# Homework 1 - Wikipedia Web Traffic Time Series

У вас есть данные по посещению 1000 страниц  Википедии из разных стран и разных девайсов ( \*  * данные взяты из [Kaggle соревнования](https://www.kaggle.com/c/web-traffic-time-series-forecasting)* )

*wikipedia_train* и *wikipedia_test* - содержат данные о трафике. Это файлы csv, где каждая строка соответствует определенной статье, и каждый столбец соответствует конкретной дате. В некоторых записях отсутствуют данные. Названия страниц содержат проект Википедии (например, en.wikipedia.org), тип доступа (например, desktop) и тип агента (например, spider). Другими словами, каждое имя статьи имеет следующий формат: «name_project_access_agent» (например, «AKB48_zh.wikipedia.org_all-access_spider»).

Вам нужно ответить на [вопросы](https://docs.google.com/forms/d/e/1FAIpQLSfDjWeeZJw5EvmKn1x_6b9xicjn7ed3MF0rbNm4Cmwr7psSkQ/viewform?usp=sf_link) и попробовать сделать самую простую модель которая сможет предсказывать будущие посещения. 

Вот примеры временных рядов посещаемости страниц Википедии (*синие* - обучающая выборка, *зеленые* - предсказания модели победителя соревнования на Kaggle, *оранжевые* - реальные значения):
![Wikipedia Web Traffic Time Series](https://image.ibb.co/cUpEJa/predictions.png)

In [70]:
import re
import pandas as pd
import numpy as np

In [71]:
train = pd.read_csv("../data/wikipedia_train.csv")
test = pd.read_csv("../data/wikipedia_test.csv")

## Data Analysis

In [72]:
def get_language(page):
    res = re.search('[a-z][a-z].wikipedia.org',page)
    if res:
        return res.group(0)[0:2]
    return 'na'

## Forecasting

Нужно преобразовать `train` данные в следующий формат:

In [73]:
train_df = train.melt(id_vars=['Page'], value_name='Visits', var_name='Date')
test_df = test.melt(id_vars=['Page'], value_name='Visits', var_name='Date')
predictions = test_df.copy()

Таким образом у вас каждая сточка содержит набор фич (`Page`, `date`) и целевую переменную (`Visits`). Преобразовать данные в такой формат поможет функция `pd.melt()` (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.melt.html)

In [74]:
train_df.head()

,Page,Date,Visits
0,15._November_de.wikipedia.org_desktop_all-agents,2015-07-01,32.0
1,2012_(film)_fr.wikipedia.org_all-access_spider,2015-07-01,2.0
2,2016_FIFA_U-20女子ワールドカップ_ja.wikipedia.org_all-a...,2015-07-01,1.0
3,2016_UEFA_Europa_League_Final_en.wikipedia.org...,2015-07-01,3.0
4,2016_in_video_gaming_en.wikipedia.org_all-acce...,2015-07-01,24.0


Оценивать качество предсказаний мы будем с помощью [SMAPE](https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error) :

In [83]:
def pandas_smape(df):
    df.fillna(0, inplace=True)
    df["SMAPE"] = 200 * np.abs(df["Visits"] - df["pred_Visits"]) / (df["Visits"] + df["pred_Visits"])
    df["SMAPE"].fillna(0, inplace=True)
    return np.mean(df["SMAPE"])

### Сколько страниц из русской Википедии в датасете?

In [76]:
train['language'] =train["Page"].apply(get_language)
print(f"Количество страниц русской википедии: {train.language.value_counts()['ru']}")

Количество страниц русской википедии: 102


### Какая самая популярная страница русской Википедии (в среднем)?

In [77]:
train['average_visits'] = train.mean(axis=1)
top = train[train.language == 'ru'].sort_values('average_visits', ascending=False).iloc[0,0]
print(f"Самая популярная страница русской википедии: {top}")

Самая популярная страница русской википедии: Facebook_ru.wikipedia.org_desktop_all-agents


In [78]:
train.drop(train.columns[-2:], inplace=True, axis=1)

### Last day baseline

Нужно сделать прогноз на основе посещений в последний известный нам день из train (продублировать значение для каждого дня в test)

In [80]:
#method1
helper = train_df.dropna().sort_values(['Page', 'Date'])\
.drop_duplicates(['Page'], keep='last').set_index('Page').transpose()
predictions['pred_Visits'] = predictions.Page.apply(lambda x: helper[x].Visits)
pandas_smape(predictions)

54.988221062619054

In [86]:
#method2
page_v = {}
p_d = train.set_index('Page').T
for page in p_d.columns:
    last_valid = p_d[page].last_valid_index()
    page_v[page] = p_d.loc[last_valid][page]
    
predictions['pred_Visits'] = predictions.Page.apply(lambda x: page_v[x])
pandas_smape(predictions)

54.988221062619054

In [87]:
#method3
helper = train.drop(train.columns[1:-1], axis=1).set_index('Page').transpose()
predictions['pred_Visits'] = predictions.Page.apply(lambda x: helper[x])
pandas_smape(predictions)

54.16127748085736

### Median baseline

Нужно сделать прогноз на основе медианы за последние **30** дней из `train`. 

А затем улучшить предсказания используя информацию выходной это или нет (воспользуйтесь функцией [dayofweek](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DatetimeIndex.dayofweek.html) ) и разные окна для подсчета медианы (7 дней, 60 дней и тд)

Вам поможет функция `pd.groupby()` (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html)

**Медиана за 30 дней**

In [81]:
train_df.Date = pd.to_datetime(train_df.Date)
train_df['dayOfWeek'] = train_df.Date.apply(lambda x: x.dayofweek)
train_df['weekend'] = train_df.dayOfWeek.apply(lambda x: 1 if x in (5, 6) else 0)
predictions.Date = pd.to_datetime(predictions.Date)
predictions['dayOfWeek'] = predictions.Date.apply(lambda x: x.dayofweek)
predictions['weekend'] = predictions.dayOfWeek.apply(lambda x: 1 if x in (5, 6) else 0)

In [88]:
helper = train_df[train_df.Date > '2016-08-01'].groupby('Page')['Visits'].median()
predictions['pred_Visits'] = predictions.Page.apply(lambda x: helper[x])
pandas_smape(predictions)

52.46588329336902

In [89]:
#не просили заміняти, це не треба
helper = train_df[train_df.Date > '2016-08-01'].fillna(0).groupby('Page')['Visits'].median()
predictions['pred_Visits'] = predictions.Page.apply(lambda x: helper[x])
pandas_smape(predictions)

51.89889768202033

**Медиана за 30 дней и выходные**

In [69]:
helper = train_df[train_df.Date > '2016-08-01'].groupby(['Page', 'weekend']).median().transpose()
predictions['pred_Visits'] = predictions[['Page','weekend']].apply(lambda x: helper[x[0]][x[1]].Visits, axis=1)
pandas_smape(predictions)

52.360885003115996

<center>**Лучшая модель!**

In [68]:
helper = train.fillna(0)
i=0
windows = [-1,-2,-3,-5,-8,-13,-21,-34,-55,-89,1,1,2,3,5,8,12,14,21,34,55,89,100,110, 200,244,360]
for w in windows:
    i+=1
    helper['pred_Visits' + str(i)] = train[train.columns[-w:]].agg(np.nanmedian, axis=1)

helper['pred_Visits'] = helper[helper.columns[-len(windows):]].median(axis=1)
helper['pred_Visits'] = helper['pred_Visits'].apply(lambda x: round(x))
helper = helper.drop(helper.columns[1:-1], axis = 1).set_index('Page').transpose()
predictions['pred_Visits'] = predictions.Page.apply(lambda x: helper[x].pred_Visits)
pandas_smape(predictions)

/home/yulia/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/numpy/lib/function_base.py:4016: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)


48.58044749458924

In [57]:
def get_weekends_workends(train):
    dates = train.set_index('Page').columns[:-2]
    dates = dates.astype('datetime64[ns]')
    weekends = []
    workends = []
    for date in dates :
        if date.dayofweek == 5 or date.dayofweek==6:
            weekends.append(date.date())
        else:
            workends.append(date.date())
    weekends = [weekend.strftime('%Y-%m-%d') for weekend in weekends]
    workends = [workend.strftime('%Y-%m-%d') for workend in workends]
    return weekends, workends

def get_days(train):
    dates = train.set_index('Page').columns[:-2]
    dates = dates.astype('datetime64[ns]')
    mn = []
    tu = []
    wd = []
    th = []
    fr = []
    st = []
    sn = []
    for date in dates :
        if date.dayofweek == 0:
            mn.append(date.date())
        elif date.dayofweek == 1:
            tu.append(date.date())
        elif date.dayofweek == 2:
            wd.append(date.date()) 
        elif date.dayofweek == 3:
            th.append(date.date())
        elif date.dayofweek == 4:
            fr.append(date.date())
        elif date.dayofweek == 5:
            st.append(date.date())
        else:
            sn.append(date.date())
    mn = [weekend.strftime('%Y-%m-%d') for weekend in mn]
    tu = [weekend.strftime('%Y-%m-%d') for weekend in tu]
    wd = [weekend.strftime('%Y-%m-%d') for weekend in wd]
    th = [weekend.strftime('%Y-%m-%d') for weekend in th]
    fr = [weekend.strftime('%Y-%m-%d') for weekend in fr]
    st = [weekend.strftime('%Y-%m-%d') for weekend in st]
    sn = [weekend.strftime('%Y-%m-%d') for weekend in sn]
    return mn, tu,wd,th,fr,st,sn

def split_df(df):
    weekends, workends = get_weekends_workends(df)
    columns_weekends = np.append('Page', weekends)
    columns_workends = np.append('Page', workends)
    return df[columns_weekends], df[columns_workends]

def split_df2(df):
    m,t,w,t1,f,s,s1 = get_days(df)
    columns_m = np.append('Page', m)
    columns_t = np.append('Page', t)
    columns_w = np.append('Page', w)
    columns_t1 = np.append('Page', t1)
    columns_f = np.append('Page', f)
    columns_s = np.append('Page', s)
    columns_s1 = np.append('Page', s1)
    return df[columns_m], df[columns_t], df[columns_w], df[columns_t1], df[columns_f], df[columns_s], df[columns_s1]

def predict(train, test):
    train.fillna(0, inplace=True)
    helper = train.copy()
    test_df = test.melt(id_vars=['Page'], value_name='Visits', var_name='Date')
    predictions = test_df.copy()
    i=0
    windows = [-1,-2,-3,-13,-21,-34,-55,-89,-100, -110, 1,1,2,3,5,8,12,14,21,34,55,89,100,110]
    for w in windows:
        i+=1
        helper['pred_Visits' + str(i)] = train[train.columns[-w:]].agg(np.nanmedian, axis=1)
    helper['pred_Visits'] = helper[helper.columns[-len(windows):]].median(axis=1)
    helper['pred_Visits'] = helper['pred_Visits'].apply(lambda x: round(x))
    helper =helper.drop(helper.columns[1:-1], axis = 1).set_index('Page').transpose()
    predictions['pred_Visits'] = predictions.Page.apply(lambda x: helper[x].pred_Visits)
    return predictions

def predict2(train, test):
    train.fillna(0, inplace=True)
    helper = train.copy()
    test_df = test.melt(id_vars=['Page'], value_name='Visits', var_name='Date')
    predictions = test_df.copy()
    i=0
    windows = [-1,-2,-3,-8,-12,-15,-21,-34,-55,-60,1,1,1,2,2,4,6,7,14,35,56,60]
    for w in windows:
        i+=1
        helper['pred_Visits' + str(i)] = train[train.columns[-w:]].agg(np.nanmedian, axis=1)
    helper['pred_Visits'] = helper[helper.columns[-len(windows):]].median(axis=1)
    helper['pred_Visits'] = helper['pred_Visits'].apply(lambda x: round(x))
    helper =helper.drop(helper.columns[1:-1], axis = 1).set_index('Page').transpose()
    predictions['pred_Visits'] = predictions.Page.apply(lambda x: helper[x].pred_Visits)
    return predictions

In [58]:
train_m, train_t, train_w, train_t1, train_f, train_s, train_s1 = split_df2(train)
test_m, test_t, test_w, test_t1, test_f, test_s, test_s1 = split_df2(test)

t_m= predict2(train_m, test_m)
t_t= predict2(train_t, test_t)
t_w= predict2(train_w, test_w)
t_t1= predict2(train_t1, test_t1)
t_f= predict2(train_f, test_f)
t_s= predict2(train_s, test_s)
t_s1= predict2(train_s1, test_s1)
n_d = pd.concat([t_m,t_t,t_w,t_t1,t_f,t_s,t_s1], axis=0)
pandas_smape(n_d)

/home/yulia/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


46.10240333664849

In [59]:
train_weekends, train_workends = split_df(train)
test_weekends, test_workends = split_df(test)

t_wrk = predict(train_workends, test_workends)
t_week = predict(train_weekends, test_weekends)
n_d = pd.concat([t_week, t_wrk], axis=0)
pandas_smape(n_d)

/home/yulia/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


46.19476571088786

P.S. Ця модель краще rolling window